# 使用xpath
XPath，全称XML Path Language，即XML路径语言，它是一门在XML文档中查找信息的语言。它最初是用来搜寻XML文档的，但是它同样适用于HTML文档的搜索。
XPath的选择功能十分强大，它提供了非常简洁明了的路径选择表达式。另外，它还提供了超过100个内建函数，用于字符串、数值、时间的匹配以及节点、序列的处理等。几乎所有我们想要定位的节点，都可以用XPath来选择

+ lxml提供了一个python的API，可以用来提取xml，html等的节点信息
+ lxml.etree, ElementTree模块
+ lxml也有其它的模块（html,csssekect等）

相关学习链接：
+ [https://www.jianshu.com/p/e084c2b2b66d]lxml的简介和中文教程
+ [https://www.w3school.com.cn/xpath/index.asp]w3school的xpath中文教程



In [2]:
from lxml import etree
# lxml是python的一个解析库，支持XPath解析方式
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text) # 声明一个解析对象
# html = etree.parse('/path/to/xx.html', etree.HTMLParser())
result = etree.tostring(html) 
# etree内会对传入的html文本自动补全，比如上面的text的最后一个li节点没有闭合，但是这里会自动补全
# tostring()方法返回的是bytes类型
# 这里还自动补全了body和html节点
print(result.decode('utf-8'))

<html><body><div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li></ul>
 </div>
</body></html>


## 所有节点
选取节点时返回的是一个列表，每个元素是`Element`类型，气候跟了节点的名称，如`html`,`body`,`div`,`ul`,`li`,`a`等

In [4]:
result = html.xpath('//*') # 选取所有节点
print(result)
print('\n')
result = html.xpath('//li') # 选取所有li节点
print(result)

[<Element html at 0x1fc4efae488>, <Element body at 0x1fc4f0d8ac8>, <Element div at 0x1fc4f0ee648>, <Element ul at 0x1fc4f0ee548>, <Element li at 0x1fc4f0eedc8>, <Element a at 0x1fc4f0ee8c8>, <Element li at 0x1fc4f0ee5c8>, <Element a at 0x1fc4f0eef48>, <Element li at 0x1fc4f0eeac8>, <Element a at 0x1fc4f0ee148>, <Element li at 0x1fc4e69c088>, <Element a at 0x1fc4e69cc48>, <Element li at 0x1fc4e69cf88>, <Element a at 0x1fc4e69ca48>]


[<Element li at 0x1fc4f0eedc8>, <Element li at 0x1fc4f0ee5c8>, <Element li at 0x1fc4f0eeac8>, <Element li at 0x1fc4e69c088>, <Element li at 0x1fc4e69cf88>]


## 子节点
+ `/` 查找节点的子节点
+ `//` 查找节点的子孙节点

In [7]:
result = html.xpath('//li/a') # 选取所有li节点的a子节点
print(result)
print('\n')

result = html.xpath('//ul//a') # 选取ul节点的a子孙节点，结果与上面的一样
print(result)
print('\n')

result = html.xpath('//ul/a') # 这里返回空列表，因为ul节点的子节点没有名称为a的节点
print(result)

[<Element a at 0x1fc4d509908>, <Element a at 0x1fc4f0d8088>, <Element a at 0x1fc4e88ea08>, <Element a at 0x1fc4e628bc8>, <Element a at 0x1fc4e945208>]


[<Element a at 0x1fc4d509908>, <Element a at 0x1fc4f0d8088>, <Element a at 0x1fc4e88ea08>, <Element a at 0x1fc4e628bc8>, <Element a at 0x1fc4e945208>]


[]


## 父节点 & 属性匹配
+ `..` 可以返回当前节点的父节点
+ 也可以用`parent::`获取父节点
+ [@href="link4.html"] 查找属性href的值为link4.html的节点
+ @class 获取class属性的值

In [9]:
result = html.xpath('//a[@href="link4.html"]/../@class')
# 选中href属性为link4.html的a节点，然后再获取其父节点，然后再获取其class属性
print(result)
print('\n')

result = html.xpath('//a[@href="link4.html"]/parent::*/@class')
print(result)

['item-1']


['item-1']


In [ ]:
## 文本获取
使用XPath中的`text()`可以获取节点中的文本

In [10]:
result = html.xpath('//li[@class="item-0"]/text()')
# 这里相当于是直接获取class=item-0的li节点自己的文本，所以输出的是最后一个li节点beietree自动补全的内容
# 因为自动补全是</li>换行了，所以输出只有一个换行符和几个空格
print(result)
print('\n')

result = html.xpath('//li[@class="item-0"]/a/text()') # 选取li节点下面a子节点的文本
print(result, '\n')

result = html.xpath('//li[@class="item-0"]//text()') # 选取所有子孙节点及自己的文本
print(result)

['\n     ']


['first item', 'fifth item'] 

['first item', 'fifth item', '\n     ']


## 属性获取
+ `@` 符号可以直接获取节点内部属性的值

In [11]:
result = html.xpath('//li/a/@href') # 获取所有a节点的href的属性值
print(result)

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']


## 属性多值匹配
有时候，某些节点的某个属性可能有多个值，例如
```
<li class="li li-first"><a href="link.html">first item</a></li>
```
这里的class属性有`li`和`li-first`两个值
此时单纯使用@class="li"则无法成功匹配。
+ 使用`contains(@属性名， 包含值)`函数可以匹配包含某个属性值的节点

In [15]:
text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[@class="li"]/a/text()')
print(result) #无法匹配
print('\n')

result = html.xpath('//li[@class="li li-first"]/a/text()')
print(result, '\n')

result = html.xpath('//li[contains(@class, "li")]/a/text()')
print(result)

[]


['first item'] 

['first item']


## 多属性匹配
另外，我们可能还遇到一种情况，那就是根据多个属性确定一个节点，这时就需要同时匹配多个属性。此时可以使用运算符and来连接。类似地，还可以用其它运算符，如
+ `or`
+ `mod` 余数
+ `|` 计算两个节点集，`//book | //cd` 返回所有拥有book和cd元素的节点集
+ `+` 加法
+ `!=` 不等于
+ `<`,`>=`


In [16]:
text = '''
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[contains(@class, "li") and @name="item"]/a/text()')
print(result)

['first item']


## 按序选择
有时候，我们在选择的时候某些属性可能同时匹配了多个节点，但是只想要其中的某个节点，如第二个节点或者最后一个节点，这时可以利用中括号传入索引的方法获取特定次序的节点

In [17]:
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li[1]/a/text()')
print(result, '\n')
result = html.xpath('//li[last()]/a/text()')
print(result, '\n')
result = html.xpath('//li[position()<3]/a/text()')
print(result, '\n')
result = html.xpath('//li[last()-2]/a/text()') # 倒数第三个
print(result, '\n')

['first item'] 

['fifth item'] 

['first item', 'second item'] 

['third item'] 



## 节点轴选择
`轴`可定义相对于当前节点的节点集
节点轴选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等

In [24]:
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html"><span>first item</span></a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li[1]/ancestor::*') # 调用ancestor轴，获取所有（*）祖先节点
print(result, '\n')
result = html.xpath('//li[1]/ancestor::div') # 调用ancestor轴，限定为名为div的节点
print(result, '\n')
result = html.xpath('//li[1]/attribute::*') # 调用attribute轴，获取所有（*）属性值
print(result, '\n')
result = html.xpath('//li[1]/child::a[@href="link1.html"]') # 调用child轴，获取所有直接子节点并选取名为a，属性href为link1.html的节点
print(result, '\n')
result = html.xpath('//li[1]/descendant::span') # 调用descendant轴，获取所有子孙节点，并返回名为span的节点
print(result, '\n')
result = html.xpath('//li[1]/following::*[2]') # 调用following轴，获取当前节点之后的所有（*）节点，并返回第二个
print(result, '\n')
result = html.xpath('//li[2]/following-sibling::*') # 调用following-sibling轴，获取当前节点后续的所有同级节点
print(result, '\n')

[<Element html at 0x1fc4e46e948>, <Element body at 0x1fc4ef06c88>, <Element div at 0x1fc4e65cf08>, <Element ul at 0x1fc4f0e9448>] 

[<Element div at 0x1fc4e65cf08>] 

['item-0'] 

[<Element a at 0x1fc4e65cf08>] 

[<Element span at 0x1fc4e71af08>] 

[<Element a at 0x1fc4e65cf08>] 

[<Element li at 0x1fc4ef06c88>, <Element li at 0x1fc4e60f608>, <Element li at 0x1fc4e90d0c8>] 

